This notebook uses GridLAB-D to study a simple tariff portfolio analysis problem. The problem is stated as follows.

Find the optimal mix of fixed and time-of-use (TOU) rate subscriptions for an unspecified feeder given the real-time wholesale market price at `SANMATO_6_N001` where 

1. the fixed price tariff generates a 100% overhead, 
2. the TOU rate generates the same revenue as the fixed price tariff
3. the TOU rate on-peak rate is twice the off-peak rate
4. the TOU rate is on-peak during the highest  consecutive hours of wholesale prices on average

First we need to download the CAISO real-time price 

In [1]:
import os,sys
for month in range(1,13):
    file = f"CAISO-SANMATO_6_N001-2021{month:02d}.csv"
    if not os.path.exists(file):
        print(f'downloading {file} CAISO...',file=sys.stderr)
        try:
            os.system(f"gridlabd market_data -m=CAISO -d=SANMATO_6_N001 -s=2021{month:02d}01 -e=2021{(month+1)%12:02d}01 > {file}")
        except:
            os.remove(file)
            raise

In [2]:
import pandas
from warnings import warn
data = []
for month in range(1,13):
    file = f'CAISO-SANMATO_6_N001-2021{month:02d}.csv'
    try:
        data.append(pandas.read_csv(file))
    except:
        warn(f"{file} load failed")
caiso = pandas.concat(data)

In [5]:
from numpy import array
from copy import copy

prices = copy(caiso)

duration = 6 # duration of peak time
overhead = 1.0 # fixed price overhead rate
fixed_cost = 100.0 # fixed cost component of price 
peak_ratio = 2.0 # onpeak/offpeak price
peak_load = 15.0 # MW
    
prices['HOUR'] = pandas.DatetimeIndex(caiso['START_TIME_LOCAL']).hour
prices['MW'] = prices['MW'] / prices['MW'].max() * peak_load
priceshape = list(prices.groupby('HOUR')['LMP'].mean().to_dict().values())
priceshape.extend(priceshape)
pricetable = list(range(24))
for h in range(24):
    pricetable[h] = sum(priceshape[h:(h+duration)])
start = pricetable.index(max(pricetable))
stop = (start + duration + 1)%24

priceshape = list(prices.groupby('HOUR')['LMP'].mean().to_dict().values())
loadshape = list((prices.groupby('HOUR')['MW'].mean()).to_dict().values())
energy_cost = sum(array(priceshape)*array(loadshape))
fixed_price = energy_cost/sum(loadshape)*(1+overhead) + fixed_cost
total_revenue = fixed_price * sum(loadshape)

print(f"Peak load....... {prices['MW'].max():10.1f} MW")
print(f"Energy cost..... ${energy_cost:10.2f}")
print(f"Total revenue... ${total_revenue:10.2f}")

onpeak_hours = list(set(range(start,stop)))
offpeak_hours = list(set(range(24)).difference(set(onpeak_hours)))

offpeak_load = sum(array(loadshape)[offpeak_hours])
onpeak_load = sum(array(loadshape)[onpeak_hours])
offpeak_price = total_revenue/(offpeak_load+peak_ratio*onpeak_load)
onpeak_price = peak_ratio*offpeak_price

with open("rate_schedules.glm","w") as fh:
    fh.write(f"""// generated by {__name__}
    schedule tou_rate {{
        * {(stop+1)%24}-{(start+1)%24} * * * {offpeak_price:.02f};
        * {start}-{stop} * * * {onpeak_price:.02f};
    }}
    schedule fixed_rate {{
        * * * * * {fixed_price:.02f};
    }}
    """)
    
!cat rate_schedules.glm

Peak load.......       15.0 MW
Energy cost..... $  10099.65
Total revenue... $  35921.56
// generated by __main__
    schedule tou_rate {
        * 1-18 * * * 172.93;
        * 17-0 * * * 345.86;
    }
    schedule fixed_rate {
        * * * * * 172.93;
    }
    

In [4]:
list(set(range(17,24)).union(set(range(15))))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23]